#Import Library

In [5]:
!pip install -q transformers sentencepiece datasets accelerate evaluate sacrebleu

import os
import numpy as np
import json
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import torch
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer

#Load model

In [6]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# model_name = "VietAI/envit5-translation"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# model.cuda()

# inputs = [
#     "vi: VietAI là tổ chức phi lợi nhuận với sứ mệnh ươm mầm tài năng về trí tuệ nhân tạo và xây dựng một cộng đồng các chuyên gia trong lĩnh vực trí tuệ nhân tạo đẳng cấp quốc tế tại Việt Nam.",
#     "vi: Theo báo cáo mới nhất của Linkedin về danh sách việc làm triển vọng với mức lương hấp dẫn năm 2020, các chức danh công việc liên quan đến AI như Chuyên gia AI (Artificial Intelligence Specialist), Kỹ sư ML (Machine Learning Engineer) đều xếp thứ hạng cao.",
#     "en: Our teams aspire to make discoveries that impact everyone, and core to our approach is sharing our research and tools to fuel progress in the field.",
#     "en: We're on a journey to advance and democratize artificial intelligence through open source and open science."
#     ]

# outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=512)
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

# # ['en: VietAI is a non-profit organization with the mission of nurturing artificial intelligence talents and building an international - class community of artificial intelligence experts in Vietnam.',
# #  'en: According to the latest LinkedIn report on the 2020 list of attractive and promising jobs, AI - related job titles such as AI Specialist, ML Engineer and ML Engineer all rank high.',
# #  'vi: Nhóm chúng tôi khao khát tạo ra những khám phá có ảnh hưởng đến mọi người, và cốt lõi trong cách tiếp cận của chúng tôi là chia sẻ nghiên cứu và công cụ để thúc đẩy sự tiến bộ trong lĩnh vực này.',
# #  'vi: Chúng ta đang trên hành trình tiến bộ và dân chủ hoá trí tuệ nhân tạo thông qua mã nguồn mở và khoa học mở.']


#Build Config

In [7]:
class BaseConfig:
    """Base Encoder Decoder config"""
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class NMTConfig(BaseConfig):
    # Data
    src_lang = 'en'
    tgt_lang = 'vi'
    max_len = 512
    add_special_tokens = True

    # Model
    model_name = "VietAI/envit5-translation"

    # Training
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    learning_rate = 5e-5
    train_batch_size = 4
    eval_batch_size = 4
    num_train_epochs = 3
    save_total_limit = 1
    ckpt_dir = './envit5-translation'
    eval_steps = 500

    # Inference
    beam_size = 5

    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

cfg = NMTConfig()
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(cfg.model_name)

# Metric
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != tokenizer.pad_token_id, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    labels = np.where(labels != tokenizer.pad_token_id, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

#Build Dataset

In [8]:
base_dir = "/content/drive/MyDrive/Machine Translation"
file_path = os.path.join(base_dir, "my_data.json")

with open(file_path, 'r') as file:
    data = json.load(file)

train_dataset, valid_dataset = train_test_split(data, test_size=0.1, random_state=42)

print("Số lượng mẫu trong tập train:", len(train_dataset))
print("Số lượng mẫu trong tập validation:", len(valid_dataset))

Số lượng mẫu trong tập train: 3757
Số lượng mẫu trong tập validation: 418


In [9]:
class CrawlDataset(Dataset):
    def __init__(self, cfg, data_list):
        super().__init__()
        self.cfg = cfg

        self.src_texts, self.tgt_texts = self.extract_texts(data_list)
        self.src_input_ids = self.texts_to_sequences(self.src_texts)
        self.labels = self.texts_to_sequences(self.tgt_texts)

    def extract_texts(self, data_list):
        src_texts = [item['en'] for item in data_list]
        tgt_texts = [item['vi'] for item in data_list]
        return src_texts, tgt_texts

    def texts_to_sequences(self, texts):
        data_inputs = self.cfg.tokenizer(
            texts,
            padding='max_length',
            truncation=True,
            max_length=self.cfg.max_len,
            return_tensors='pt'
        )
        return data_inputs.input_ids

    def __getitem__(self, idx):
        return {
            "input_ids": self.src_input_ids[idx],
            "labels": self.labels[idx]
        }

    def __len__(self):
        return len(self.src_input_ids)

In [10]:
train_dataset = CrawlDataset(cfg, train_dataset)
valid_dataset = CrawlDataset(cfg, valid_dataset)

#Trainer

In [11]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    save_strategy='steps',
    save_steps=cfg.eval_steps,
    eval_steps=cfg.eval_steps,
    output_dir=cfg.ckpt_dir,
    per_device_train_batch_size=cfg.train_batch_size,
    per_device_eval_batch_size=cfg.eval_batch_size,
    learning_rate=cfg.learning_rate,
    save_total_limit=cfg.save_total_limit,
    num_train_epochs=cfg.num_train_epochs,
    load_best_model_at_end=True,
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=cfg.tokenizer,
    model=model
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=cfg.tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss,Bleu,Gen Len
500,1.790000,0.040382,11.652400,6.330100
1000,0.032000,0.037792,14.080200,6.409100
1500,0.019600,0.036855,15.278300,6.440200
2000,0.014700,0.038308,15.680500,6.514400
2500,0.011000,0.038616,15.342500,6.413900


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=2820, training_loss=0.332143150576463, metrics={'train_runtime': 4463.9687, 'train_samples_per_second': 2.525, 'train_steps_per_second': 0.632, 'total_flos': 8194422043312128.0, 'train_loss': 0.332143150576463, 'epoch': 3.0})

In [12]:
inputs = [
    "vi: VietAI là tổ chức phi lợi nhuận với sứ mệnh ươm mầm tài năng về trí tuệ nhân tạo và xây dựng một cộng đồng các chuyên gia trong lĩnh vực trí tuệ nhân tạo đẳng cấp quốc tế tại Việt Nam.",
    "en: Pancreatic carcinoma is a hypovascular tumor and is best detected in the late arterial phase at 35-40 sec p.i. when the normal glandular tissue enhances optimally and the hypovascular tumor does not.",
    ]

outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=512)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['en: VietAI is a non-profit organization whose mission is to cultivate talent in artificial intelligence and to build an international-level community of artificial intelligence professionals in Vietnam.', 'vi: Ung thư tụy là một khối u thiếu mạch và được phát hiện rõ nhất ở giai đoạn động mạch muộn 35-40 giây sau tiêm. Khi đó mô tuyến tăng quang tối ưu và khối u thiếu mạch thì không.']


In [14]:
trainer.save_model("/content/drive/MyDrive/Machine Translation/model")